# Esercitazione 1

## Conceptual Similarity with WordNet

In [68]:
import math
import pandas as pd
from scipy import stats
import string
import nltk
from nltk.corpus import semcor, stopwords
from nltk.corpus import wordnet as wn
from operator import itemgetter
import random
from tqdm import tqdm

Come prima operazione carichiamo in memoria il dataset fornito.

Questo file csv contiene le coppie di termini e la similarità annotata (gold-standard).

In [51]:
dataset = pd.read_csv('./01-WordSim353.csv')

Durante l'esercitazione verranno utilizzate tre differenti metriche per misurare la similarità tra sensi.

### Wu & Palmer

La misura di similarità di Wu & Palmer sfrutta la struttura ad albero di WordNet per calcolare la prossimità tra due sensi dati in input.

$$sim_{wu-palmer}(s_1, s_2) = \frac{2 \cdot depth(LCS)}{depth(s_1) + depth(s_2)}$$

Dove $LCS$ è il lowest common subsumer dei due sensi passati per argomento.

In [52]:
def lcs(s1, s2):
    lcs, h1, h2 = None, [s1], [s2]
    for synset in h1:
        s_hypernyms = synset.hypernyms()
        if len(s_hypernyms) > 0:
            h1.extend(s_hypernyms)
    for synset in h2:
        s_hypernyms = synset.hypernyms()
        if len(s_hypernyms) > 0:
            h2.extend(s_hypernyms)
    if len(h1) > 0 and len(h2) > 0:
        common = set(h1).intersection(set(h2))
        if len(common) > 0:
            common = [(s, s.max_depth() + 1) for s in common]
            return sorted(common, key=itemgetter(1), reverse=True)[0][0]
    return None


Questa funzione lambda si occupa di ricavare la profondità di un senso passato in input.

In [53]:
depth = lambda synset: max([len(path) for path in synset.hypernym_paths()])

In [54]:
def sim_wu_palmer(s1, s2):
    lcs_synset = lcs(s1, s2)
    lcs_depth = lcs_synset.max_depth() + 1 if lcs_synset is not None else 0
    d1 = depth(s1)
    d2 = depth(s2)
    if d1 > 0 or d2 > 0:
        return (2 * lcs_depth) / (d1 + d2)
    return 1

### Shortest Path

La misura Shortest Path calcola la lunghezza del cammino tra i due nodi del grafo di WordNet corrispondenti ai *synset*.

$$sim_{path}(s_1, s_2) = 2 \cdot depthMax - len(s_1, s_2)$$

Se la lunghezza del cammino è 0, ovvero i sensi sono sovrapposti, la similarità è massima. Quindi possiamo dire che è pari a 2*depthMax, ovvero due volte la massima profondità di WordNet.

Per calcolare le due misure di similarità successive occorre la massima profondità dell'albero di WordNet.

In [55]:
depth_max = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

Il metodo *min_len_path* calcola la distanza del percorso tra i due sensi. Tra i possibili cammini percorribili si sceglie quello passante per $LCS$, per cui forziamo il calcolo del percorso di lunghezza minima.

In [56]:
def min_len_path(s1, s2):
    distance = 0
    lcs_synset = lcs(s1, s2)
    if lcs_synset is not None:
        distance = (depth(s1) - depth(lcs_synset)) - (depth(s2) - depth(lcs_synset))
    return distance


In [57]:
def sim_shortest_path(s1, s2):
    min_path = 0
    len_path = min_len_path(s1, s2)
    if len_path >= 0:
        min_path = 2 * depth_max - len_path
    return min_path

### Lealcock & Chodorow

Questa misura di similarità assume valori nell'intervallo (0, log(2*depthMax + 1)), l'aggiunta di 1 nell'argomento del logaritmo è per evitare lo zero.

$$sim_{LC}(s_1, s_2) = - log \frac{len(s_1, s_2)}{2 \cdot depthMax}$$

In [58]:
def sim_leakcock_chodorow(s1, s2):
    sim = 0
    len_path = min_len_path(s1, s2)
    if len_path > 0:
        sim = -math.log(len_path / (2 * depth_max))
    return sim

Per praticità, nei test raccogliamo in una lista le coppie (funzione, nome) per le tre misure di similarità.

In [59]:
similarities = [(sim_wu_palmer, 'Wu Palmer'),
 (sim_shortest_path, 'Shortest path'),
 (sim_leakcock_chodorow, 'Leakcock Chodorow')]

Tutti i metodi sopra esposti hanno come argomento due sensi, tuttavia nel dataset preso in esame sono annotati dei termini.

Per risolvere il problema, si considera un termine come il minimo contesto necessario e sufficiente per disambiguare l'altro membro della coppia.

La similarità tra due termini, indipendentemente dal metodo utilizzato, sarà quindi il massimo tra ogni possibile coppia di sensi rappresentata dai due termini.

In [60]:
def compute_similarity(t1, t2, sim_func):
    synsets1 = wn.synsets(t1)
    synsets2 = wn.synsets(t2)
    max_sim = 0.0
    best_pair = None
    for s1 in synsets1:
        for s2 in synsets2:
            sim = sim_func(s1, s2)
            if sim >= max_sim:
                max_sim = sim
                best_pair = (s1, s2)
    return best_pair, max_sim

Eseguiamo quindi le tre misure di similarità sull'intero dataset, calcolando la correlazione tra le similarità ottenute e quelle annotate tramite gli indici di Pearson e di Spearman.

In [61]:
for sim_func, sim_name in similarities:
    expected, obtained = [], []
    print('\n' + sim_name)
    for idx, row in dataset.iterrows():
        t1, t2 = row['Word 1'], row['Word 2']
        expected_sim = row['Human (mean)']
        sense_pair, similarity = compute_similarity(t1, t2, sim_func)
        if sense_pair is None:
            print(f'\tUnable to compute best senses pair between {t1} and {t2}')
        else:
            expected.append(expected_sim)
            obtained.append(similarity)
    print(stats.pearsonr(expected, obtained))
    print(stats.spearmanr(expected, obtained))


Wu Palmer
	Unable to compute best senses pair between Maradona and football
PearsonRResult(statistic=0.26572842333653923, pvalue=4.214296990970517e-07)
SignificanceResult(statistic=0.324400489230381, pvalue=4.5457429629669424e-10)

Shortest path
	Unable to compute best senses pair between Maradona and football
PearsonRResult(statistic=-0.047634998203739294, pvalue=0.3729083492751214)
SignificanceResult(statistic=-0.042286502469992174, pvalue=0.42900411307508346)

Leakcock Chodorow
	Unable to compute best senses pair between Maradona and football
PearsonRResult(statistic=-0.10971193205470338, pvalue=0.03966018541582094)
SignificanceResult(statistic=-0.0992681329962518, pvalue=0.06282779628379778)


I dati presenti nel gold-standard non risultano altamente correlati con le similarità ricavate tramite le tre misure proposte. Questo fenomeno è spiegato dal fatto che gli approcci proposti non sono particolarmente raffinati. Inoltre, sfruttando in modo preponderante la struttura ad albero di WordNet, risentono dei problemi intrinseci della risorsa ovvero del fatto che i sensi non siano equamente densi su tutto il grafo.

In tutte le misure testate, la coppia di termini (Maradona - football) non ha prodotto risultati. Questo è in linea con quanto atteso in quanto WordNet non indicizza persone fisiche tra i suoi sensi.

## Word Sense Disambiguation

Per questa parte dell'esercitazione si richiede di disambiguare i sensi partendo dai termini di un contesto.

Il dataset utilizzato per questo problema è SemCor, scaricabile tramite la libreria nltk.

In [69]:
print('Getting SemCor, this may take a while...')
semcor_corpus = semcor.sents()
semcor_tags = semcor.tagged_sents(tag="sem")

Getting SemCor, this may take a while...


Per evitare di aggiungere rumore durante la computazione, è utile eliminare le stopwords dalle frasi analizzate.

In [63]:
def remove_stopwords(phrase):
    phrase = phrase.split()
    for p in string.punctuation:
        phrase = {item.replace(p, '') for item in phrase}
    phrase = {item.replace('\'s', '') for item in phrase}
    stop = stopwords.words('english')
    return {t for t in phrase if t not in stop}

Inoltre, per evitare estrazioni di contesti particolarmente difficili o particolarmente semplici verranno estratte frasi dal dataset in modo casuale.

Il metodo *get_sentence_from_semcor* permette di estrarre la tripla (parola, frase/contesto, senso_target).

In [64]:
def get_sentence_from_semcor(sentence, tags):
    for curr_word in range(len(tags)):
        if isinstance(tags[curr_word], nltk.Tree) and \
                isinstance(tags[curr_word][0], str) and \
                isinstance(tags[curr_word].label(), nltk.corpus.reader.wordnet.Lemma):
            word = tags[curr_word][0]
            target = tags[curr_word].label().synset()
            if target.pos() == 'n':
                return word, sentence, target
    return None

Il metodo *get_random_sentences* si occupa di estrarre 50 frasi del dataset. Le frasi estratte sono salvate in un insieme comune che non permette di estrarre la stessa frase una seconda volta.

In questo modo si aumenta la varietà dei dati analizzati.

In [65]:
selected_idx = {-1}


def get_random_sentences(corpus, tags):
    max_n = len(corpus)
    sentences = []
    while len(sentences) < 50:
        if len(selected_idx) == max_n:
            raise Exception('Not enough data in remaining in the corpus')
        idx = -1
        while idx in selected_idx:
            idx = random.randint(0, max_n - 1)
        computed_tuple = get_sentence_from_semcor(corpus[idx], tags[idx])
        selected_idx.add(idx)
        if computed_tuple is not None:
            sentences.append(computed_tuple)
    return sentences

Per disambiguare i termini è stato utilizzato l'algoritmo Lesk con approccio bag-of-words.

In [66]:
def lesk(word, sentence):
    synset = wn.synsets(word)
    if synset is None or len(synset) == 0:
        return None
    guess = synset[0]
    max_overlap = 0
    for sense in synset:
        signature = remove_stopwords(sense.definition())
        for ex in sense.examples():
            signature.union(remove_stopwords(ex))
        overlap = len(signature.intersection(sentence))
        if overlap > max_overlap:
            max_overlap = overlap
            guess = sense
    return guess

Infine, si esegue l'algoritmo e si calcola l'accuratezza media rispetto ai dati annotati.

In [67]:
acc = []
for i in tqdm(range(10)):
    corpus_sentences = get_random_sentences(semcor_corpus, semcor_tags)
    corpus_sentences = [(x[0], remove_stopwords(str(x[1])), x[2]) for x in corpus_sentences]
    tp = 0
    for s in corpus_sentences:
        best_sense = lesk(s[0], s[1])
        if best_sense is not None:
            if best_sense == s[2]:
                tp += 1
    acc.append(float(tp*100)/float(len(corpus_sentences)))
print(f'Avg accuracy: {sum(acc)/len(acc):.2f}%')

100%|██████████| 10/10 [00:22<00:00,  2.27s/it]

Avg accuracy: 50.80%


In media l'accuratezza ottenuta oscilla tra il 40% e il 50%. Per ottenere risultati migliori sarebbe necessario implementare la versione dell'algoritmo *corpus-Lesk*, in modo da riuscire a gestire sensi e non termini durante la computazione. Un ulteriore raffinamento potrebbe essere quello di espandere le frasi passate in input in modo da rendere più significativo l'overlap.